<a href="https://colab.research.google.com/github/chqzeng/WaterSatOnCloud/blob/main/Tool4%20-%20Rayleigh%20Atmospheric%20Correction/Tool4_Rayleigh_correction_2_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Py6S for Rayleigh Correction of level-1 LST8 data

Here we generate a lookup table to do a simple Rayleigh atmospheric correction for pixel values extracted in Tool 2.

Atmospheric reflectance is calculated in the absence of aerosol, and is removed from TOA reflectance according to the solar/observation geometry.

## Step 2 - Applying the Lookup Table to TOA Reflectance

In [1]:
# Load libaries
import pandas as pd
import numpy as np
from scipy.interpolate import RegularGridInterpolator
pd.options.mode.chained_assignment = None  # default='warn'

# Load Lookup Table

In [2]:
# Load the LUT from the GitHub folder
url = 'https://github.com/chqzeng/WaterSatOnCloud/blob/main/Tool4%20-%20Rayleigh%20Atmospheric%20Correction/LUT.csv?raw=true'
LUT = pd.read_csv(url)
LUT

,Band,SZA,VZA,RAA,R_atm
0,B1,0,0,0,0.091
1,B1,0,0,10,0.091
2,B1,0,0,20,0.091
3,B1,0,0,30,0.091
4,B1,0,0,40,0.091
...,...,...,...,...,...
13846,B9,80,80,140,0.023
13847,B9,80,80,150,0.025
13848,B9,80,80,160,0.026
13849,B9,80,80,170,0.028


In [3]:
# Turn the 2D LUT into a 3D numpy array with SZA, VZA and RAA as dimensions
ac_lut = LUT
ac_lut = ac_lut[~ac_lut.isin([np.nan]).any(1)].reset_index(drop=True)
bands = ac_lut.Band.unique()

# solar and viewing zenith angles
my_SZAs = list(range(0, 90, 10))
my_VZAs = list(range(0, 90, 10))

# relative azimuth
my_RAAs = list(range(0, 190, 10))

ac_lut_array = {}

for band in bands:
    df_band = ac_lut[ac_lut.Band == band]
    array_SZA = np.zeros((9,9,19))

    # SZA loop
    for i in range(9):
        array_VZA = np.zeros((9,19))

        # VZA loop
        for ii in range(9):
            array_RAA  = df_band.R_atm[(df_band.SZA==my_SZAs[i]) & (df_band.VZA==my_VZAs[ii])].values
            array_VZA[ii,:] = array_RAA

        array_SZA[i,:,:] = array_VZA
    ac_lut_array[band] = array_SZA

<ipython-input-3-1db2c248685c>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  ac_lut = ac_lut[~ac_lut.isin([np.nan]).any(1)].reset_index(drop=True)


# Sample Data

In [4]:
# The output data from Tool2 - GEE LST8 Matchup Extraction - Level 1. This can be replaced by output data with user-defined locations and time.

data = ("""index,B1,B10,B11,B2,B3,B4,B5,B6,B7,B8,B9,QA_PIXEL,QA_RADSAT,SAA,SZA,VAA,VZA,is_water,latitude,longitude
0,0.6378574967384338,270.9753112792969,271.2830505371094,0.6193544864654541,0.6119202971458435,0.6139735579490662,0.6753119230270386,0.38771313428878784,0.26201555132865906,0.5960133671760559,0.01449086144566536,22280.0,0.0,14024.0,3213.0,-7289.0,455.0,0.0,39.474744,-86.898353
1,0.11458732932806015,296.55841064453125,294.0486755371094,0.08930002897977829,0.06762196123600006,0.04144937917590141,0.3036518692970276,0.12595979869365692,0.0464886799454689,0.057929251343011856,0.0011576770339161158,21824.0,0.0,13016.0,2792.0,-8973.0,360.0,0.0,35.98,-78.83941
2,0.11803778260946274,304.1044006347656,302.8207092285156,0.09954726696014404,0.07926090806722641,0.06544844806194305,0.07127939164638519,0.04090527817606926,0.03108357824385166,0.07438331097364426,0.0010863732313737273,21824.0,0.0,12697.0,2553.0,-8933.0,366.0,0.0,38.04947,-99.827""")

df = pd.DataFrame([data.split(',') for data in data.split('\n')[1:]], columns=[data for data in data.split('\n')[0].split(',')])

print(df)

  index                   B1                 B10                B11  \
0     0   0.6378574967384338   270.9753112792969  271.2830505371094   
1     1  0.11458732932806015  296.55841064453125  294.0486755371094   
2     2  0.11803778260946274   304.1044006347656  302.8207092285156   

                    B2                   B3                   B4  \
0   0.6193544864654541   0.6119202971458435   0.6139735579490662   
1  0.08930002897977829  0.06762196123600006  0.04144937917590141   
2  0.09954726696014404  0.07926090806722641  0.06544844806194305   

                    B5                   B6                   B7  ...  \
0   0.6753119230270386  0.38771313428878784  0.26201555132865906  ...   
1   0.3036518692970276  0.12595979869365692   0.0464886799454689  ...   
2  0.07127939164638519  0.04090527817606926  0.03108357824385166  ...   

                      B9 QA_PIXEL QA_RADSAT      SAA     SZA      VAA    VZA  \
0    0.01449086144566536  22280.0       0.0  14024.0  3213.0  -7289.0

In [5]:
# Calculate angles in degrees
df['SZA_degree'] = df.SZA.astype(float) * 0.01
df['SAA_degree'] = df.SAA.astype(float) * 0.01
df['VZA_degree'] = df.VZA.astype(float) * 0.01
df['VAA_degree'] = df.VAA.astype(float) * 0.01
df['RAA_degree'] = (df.SAA_degree.astype(float)  - df.VAA_degree.astype(float)).abs()

# If angle larger than 180, make it 360 - angle
df.RAA_degree[df['RAA_degree'] > 180] = 360 - df.RAA_degree[df['RAA_degree'] > 180]

print(df)

  index                   B1                 B10                B11  \
0     0   0.6378574967384338   270.9753112792969  271.2830505371094   
1     1  0.11458732932806015  296.55841064453125  294.0486755371094   
2     2  0.11803778260946274   304.1044006347656  302.8207092285156   

                    B2                   B3                   B4  \
0   0.6193544864654541   0.6119202971458435   0.6139735579490662   
1  0.08930002897977829  0.06762196123600006  0.04144937917590141   
2  0.09954726696014404  0.07926090806722641  0.06544844806194305   

                    B5                   B6                   B7  ...  \
0   0.6753119230270386  0.38771313428878784  0.26201555132865906  ...   
1   0.3036518692970276  0.12595979869365692   0.0464886799454689  ...   
2  0.07127939164638519  0.04090527817606926  0.03108357824385166  ...   

       VAA    VZA is_water   latitude   longitude SZA_degree SAA_degree  \
0  -7289.0  455.0      0.0  39.474744  -86.898353      32.13     140.24   

# Interpolating Atmospheric Reflectance

In [6]:
R_atm = pd.DataFrame()

for band in bands:

    interp = RegularGridInterpolator((my_SZAs, my_VZAs, my_RAAs), ac_lut_array[band])
    R_atm_band = interp( df[['SZA_degree', 'VZA_degree','RAA_degree']].values )
    R_atm[band] = R_atm_band

print(R_atm)

         B1        B2        B3        B4     B5   B6   B7        B8     B9
0  0.088741  0.064326  0.032206  0.017545  0.006  0.0  0.0  0.028661  0.001
1  0.089272  0.064501  0.032280  0.017640  0.006  0.0  0.0  0.028640  0.001
2  0.089259  0.064356  0.032268  0.017634  0.006  0.0  0.0  0.028634  0.001


# Rayleigh Correction and Viewing Results

In [7]:
# Atmopsheric Correction: remove Rayleigh reflectance from TOA reflectance
for band in bands:
    df[band] = df[band].astype(float) - R_atm[band]
print(df)

  index        B1                 B10                B11        B2        B3  \
0     0  0.549116   270.9753112792969  271.2830505371094  0.555028  0.579714   
1     1  0.025316  296.55841064453125  294.0486755371094  0.024799  0.035342   
2     2  0.028779   304.1044006347656  302.8207092285156  0.035192  0.046993   

         B4        B5        B6        B7  ...      VAA    VZA is_water  \
0  0.596429  0.669312  0.387713  0.262016  ...  -7289.0  455.0      0.0   
1  0.023809  0.297652  0.125960  0.046489  ...  -8973.0  360.0      0.0   
2  0.047814  0.065279  0.040905  0.031084  ...  -8933.0  366.0      0.0   

    latitude   longitude SZA_degree SAA_degree VZA_degree VAA_degree  \
0  39.474744  -86.898353      32.13     140.24       4.55     -72.89   
1      35.98   -78.83941      27.92     130.16       3.60     -89.73   
2   38.04947     -99.827      25.53     126.97       3.66     -89.33   

  RAA_degree  
0     146.87  
1     140.11  
2     143.70  

[3 rows x 26 columns]
